In [1]:
import torch
import pickle
import numpy as np
import pandas as pd
import os

from os.path import dirname



root_path = dirname(os.getcwd()) + "/HGNN_NA"

pd.set_option("display.max_columns", None)
data_dir = root_path + "/data/datasets/original/"
data_dir_processed = root_path + "/data/datasets/processed/"
data_dir_graphs = root_path + "/data/datasets/graphs/"

print(root_path, data_dir, data_dir_processed, data_dir_graphs, sep="\n")

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = "cpu"

/home/sebdis/ProcessMining/HGNN/HGNN_NA
/home/sebdis/ProcessMining/HGNN/HGNN_NA/data/datasets/original/
/home/sebdis/ProcessMining/HGNN/HGNN_NA/data/datasets/processed/
/home/sebdis/ProcessMining/HGNN/HGNN_NA/data/datasets/graphs/


In [2]:
dataset = "BPI Challenge 2017 - Offer log"

In [3]:
categorical_columns = [
    "Action",
    "org:resource",
    "Activity",
    "EventOrigin",
    "lifecycle:transition",
    "case:Selected",
    "case:Accepted",
    "org:resource:role"
]
real_value_columns = [
    "time:timestamp",
    "case:MonthlyCost",
    "case:FirstWithdrawalAmount",
    "case:CreditScore",
    "case:OfferedAmount",
    "case:NumberOfTerms",
]

In [4]:
tab_all = pd.read_csv(data_dir_processed+dataset+"_processed_all.csv")
tab_all.head()

,Action,org:resource,Activity,EventOrigin,lifecycle:transition,time:timestamp,CaseID,case:MonthlyCost,case:Selected,case:FirstWithdrawalAmount,case:CreditScore,case:OfferedAmount,case:NumberOfTerms,case:Accepted,org:resource:role
0,Created,User_17,O_Create Offer,Offer,complete,0.000000,Offer_247135719,5.307079,False,9.210340,0.0,9.21034,4.043051,True,Role 1
1,statechange,User_17,O_Created,Offer,complete,0.094048,Offer_247135719,5.307079,False,9.210340,0.0,9.21034,4.043051,True,Role 1
2,statechange,User_17,O_Sent (online only),Offer,complete,1.591815,Offer_247135719,5.307079,False,9.210340,0.0,9.21034,4.043051,True,Role 1
3,statechange,User_17,O_Cancelled,Offer,complete,1.716411,Offer_247135719,5.307079,False,9.210340,0.0,9.21034,4.043051,True,Role 1
4,Created,User_17,O_Create Offer,Offer,complete,1.727046,Offer_941964966,5.307079,False,8.318742,0.0,9.21034,4.043051,True,Role 1


In [5]:
import random

torch.manual_seed(0)
torch.cuda.manual_seed(0)
random.seed(0)
np.random.seed(0)

In [6]:
with open(data_dir_graphs + dataset + "_TRAIN_event_prediction_FINAL.pkl", "rb") as f:
    X_train = pickle.load(f)
with open(data_dir_graphs + dataset + "_VALID_event_prediction_FINAL.pkl", "rb") as f:
    X_valid = pickle.load(f)
with open(data_dir_graphs + dataset + "_TEST_event_prediction_FINAL.pkl", "rb") as f:
    X_test = pickle.load(f)

In [7]:

from torch_geometric.data import Dataset
from torch_geometric.loader import DataLoader
from torch_geometric.transforms import ToUndirected, NormalizeFeatures

transform = ToUndirected()

with torch.no_grad():
        for i in range(len(X_train)):
                X_train[i] = transform(X_train[i])
        for i in range(len(X_valid)):
                X_valid[i] = transform(X_valid[i])
        for i in range(len(X_test)):
                X_test[i] = transform(X_test[i])
    






In [8]:
edge_types = set()
node_types = set()
for i in range(len(X_train)):
    n, edge_type = X_train[i].metadata()
    for x in n:
        node_types.add(x)
    for x in edge_type:
        edge_types.add(x)
for i in range(len(X_valid)):
    n, edge_type = X_valid[i].metadata()
    for x in n:
        node_types.add(x)
    for x in edge_type:
        edge_types.add(x)
for i in range(len(X_test)):
    n, edge_type = X_test[i].metadata()
    for x in n:
        node_types.add(x)
    for x in edge_type:
        edge_types.add(x)



In [9]:
node_types = list(node_types)
edge_types = list(edge_types)

In [10]:
node_types

['case:CreditScore',
 'lifecycle:transition',
 'time:timestamp',
 'case:FirstWithdrawalAmount',
 'EventOrigin',
 'case:MonthlyCost',
 'case:Accepted',
 'case:Selected',
 'case:OfferedAmount',
 'org:resource',
 'org:resource:role',
 'Action',
 'case:NumberOfTerms',
 'Activity']

In [11]:
edge_types

[('case:MonthlyCost', 'rev_related_to', 'Activity'),
 ('Activity', 'related_to', 'case:NumberOfTerms'),
 ('time:timestamp', 'related_to', 'time:timestamp'),
 ('case:Selected', 'rev_related_to', 'Activity'),
 ('case:Accepted', 'rev_related_to', 'Activity'),
 ('Activity', 'related_to', 'case:OfferedAmount'),
 ('EventOrigin', 'rev_related_to', 'Activity'),
 ('org:resource:role', 'related_to', 'org:resource:role'),
 ('Activity', 'followed_by', 'Activity'),
 ('Activity', 'related_to', 'case:CreditScore'),
 ('Activity', 'related_to', 'EventOrigin'),
 ('org:resource:role', 'rev_related_to', 'Activity'),
 ('Activity', 'related_to', 'org:resource:role'),
 ('Activity', 'related_to', 'lifecycle:transition'),
 ('org:resource', 'related_to', 'org:resource'),
 ('Activity', 'related_to', 'org:resource'),
 ('case:CreditScore', 'rev_related_to', 'Activity'),
 ('Activity', 'related_to', 'case:FirstWithdrawalAmount'),
 ('case:FirstWithdrawalAmount', 'rev_related_to', 'Activity'),
 ('case:OfferedAmount', 

In [12]:
def get_weights(load, key):
    weights = []
    
    cl_train = [0 for _ in tab_all[key].unique()]
    
    print(cl_train)
    
    for i,x in enumerate(load):

        
        classes = x.y[key]

        # print(classes)
        
        for c in list(classes):
            try:
                cl_train[c] +=1
            except KeyError:
                cl_train[c] = 1
    s = sum(cl_train)
    
    print(cl_train)
    
    weights = [s/x if x > 0 else 0 for x in cl_train]

    # weights = [0.7,0.7,1,0.7,0.7,0.7,0.7,0.7,0.7,0.7]
    weights = torch.tensor(weights, device=device)
    print(weights)
    return weights
    

In [13]:
edge_features_dims = {}

for i in range(len(X_train)):
    for k in X_train[i].edge_attr_dict.keys():
        edge_features_dims[k] = X_train[i].edge_attr_dict[k].shape[1]
for i in range(len(X_valid)):
    for k in X_train[i].edge_attr_dict.keys():
        edge_features_dims[k] = X_train[i].edge_attr_dict[k].shape[1]
for i in range(len(X_test)):
    for k in X_train[i].edge_attr_dict.keys():
        edge_features_dims[k] = X_train[i].edge_attr_dict[k].shape[1]

edge_features_dims

{('Action', 'related_to', 'Action'): 2,
 ('Activity', 'followed_by', 'Activity'): 2,
 ('time:timestamp', 'related_to', 'time:timestamp'): 2,
 ('org:resource', 'related_to', 'org:resource'): 2,
 ('org:resource:role', 'related_to', 'org:resource:role'): 2}

In [14]:
act_weights = get_weights(DataLoader(X_train, batch_size=1024, shuffle=False), "Activity")
# res_roles_weights = get_weights(DataLoader(X_train, batch_size=1024, shuffle=False), "org:resource:role")


[0, 0, 0, 0, 0, 0, 0, 0]
[9050, 11436, 0, 0, 2444, 12248, 21275, 991]
tensor([ 6.3474,  5.0231,  0.0000,  0.0000, 23.5041,  4.6901,  2.7001, 57.9657],
       device='cuda:0')


In [15]:
from data import utils

one_hot_resource = utils.get_one_hot_encoder(tab_all, "org:resource")
one_hot_resource_role = utils.get_one_hot_encoder(tab_all, "org:resource:role")
resources = tab_all["org:resource"]
resources_role = tab_all["org:resource:role"]
map_resource_to_role = {}
for i in range(len(resources)):
    map_resource_to_role[resources[i]] = resources_role[i]

map_resource_to_role

{'User_17': 'Role 1',
 'User_1': 'Role 3',
 'User_116': 'Role 2',
 'User_95': 'Role 2',
 'User_29': 'Role 2',
 'User_119': 'Role 2',
 'User_38': 'Role 1',
 'User_68': 'Role 2',
 'User_19': 'Role 1',
 'User_90': 'Role 2',
 'User_52': 'Role 1',
 'User_117': 'Role 2',
 'User_115': 'Role 2',
 'User_3': 'Role 1',
 'User_112': 'Role 2',
 'User_87': 'Role 2',
 'User_113': 'Role 2',
 'User_27': 'Role 2',
 'User_99': 'Role 2',
 'User_118': 'Role 2',
 'User_8': 'Role 1',
 'User_75': 'Role 2',
 'User_120': 'Role 2',
 'User_16': 'Role 1',
 'User_43': 'Role 1',
 'User_89': 'Role 1',
 'User_15': 'Role 1',
 'User_28': 'Role 1',
 'User_20': 'Role 1',
 'User_7': 'Role 1',
 'User_2': 'Role 1',
 'User_30': 'Role 2',
 'User_114': 'Role 2',
 'User_14': 'Role 1',
 'User_109': 'Role 2',
 'User_9': 'Role 1',
 'User_13': 'Role 1',
 'User_24': 'Role 1',
 'User_31': 'Role 1',
 'User_100': 'Role 2',
 'User_127': 'Role 2',
 'User_5': 'Role 1',
 'User_10': 'Role 1',
 'User_70': 'Role 1',
 'User_22': 'Role 1',
 'Use

In [16]:
map_resource_index = {}
for x in tab_all["org:resource"].unique():
    map_resource_index[x] = torch.argmax(torch.tensor(utils.get_one_hot_encodings(one_hot_resource, np.array([x])))).item()
map_resource_index

{'User_17': 53,
 'User_1': 0,
 'User_116': 19,
 'User_95': 139,
 'User_29': 66,
 'User_119': 22,
 'User_38': 76,
 'User_68': 109,
 'User_19': 55,
 'User_90': 134,
 'User_52': 92,
 'User_117': 20,
 'User_115': 18,
 'User_3': 67,
 'User_112': 15,
 'User_87': 130,
 'User_113': 16,
 'User_27': 64,
 'User_99': 143,
 'User_118': 21,
 'User_8': 122,
 'User_75': 117,
 'User_120': 24,
 'User_16': 52,
 'User_43': 82,
 'User_89': 132,
 'User_15': 51,
 'User_28': 65,
 'User_20': 57,
 'User_7': 111,
 'User_2': 56,
 'User_30': 68,
 'User_114': 17,
 'User_14': 45,
 'User_109': 11,
 'User_9': 133,
 'User_13': 34,
 'User_24': 61,
 'User_31': 69,
 'User_100': 2,
 'User_127': 31,
 'User_5': 89,
 'User_10': 1,
 'User_70': 112,
 'User_22': 59,
 'User_11': 12,
 'User_49': 88,
 'User_92': 136,
 'User_85': 128,
 'User_4': 78,
 'User_41': 80,
 'User_71': 113,
 'User_21': 58,
 'User_37': 75,
 'User_110': 13,
 'User_44': 83,
 'User_34': 72,
 'User_74': 116,
 'User_36': 74,
 'User_18': 54,
 'User_12': 23,
 'User_

In [17]:
map_resource_role_index = {}
for x in tab_all["org:resource:role"].unique():
    map_resource_role_index[x] = torch.argmax(torch.tensor(utils.get_one_hot_encodings(one_hot_resource_role, np.array([x])))).item()
map_resource_role_index

{'Role 1': 0, 'Role 3': 2, 'Role 2': 1}

In [18]:
map_resource_to_role = { map_resource_index[k] : map_resource_role_index[v] for k,v in map_resource_to_role.items()}
map_resource_to_role

{53: 0,
 0: 2,
 19: 1,
 139: 1,
 66: 1,
 22: 1,
 76: 0,
 109: 1,
 55: 0,
 134: 1,
 92: 0,
 20: 1,
 18: 1,
 67: 0,
 15: 1,
 130: 1,
 16: 1,
 64: 1,
 143: 1,
 21: 1,
 122: 0,
 117: 1,
 24: 1,
 52: 0,
 82: 0,
 132: 0,
 51: 0,
 65: 0,
 57: 0,
 111: 0,
 56: 0,
 68: 1,
 17: 1,
 45: 0,
 11: 1,
 133: 0,
 34: 0,
 61: 0,
 69: 0,
 2: 1,
 31: 1,
 89: 0,
 1: 0,
 112: 0,
 59: 0,
 12: 0,
 88: 0,
 136: 0,
 128: 0,
 78: 0,
 80: 0,
 113: 0,
 58: 0,
 75: 0,
 13: 0,
 83: 0,
 72: 0,
 116: 0,
 74: 0,
 54: 0,
 23: 0,
 10: 0,
 3: 1,
 100: 0,
 86: 0,
 73: 0,
 90: 0,
 62: 0,
 4: 1,
 71: 0,
 50: 1,
 63: 0,
 60: 0,
 115: 0,
 29: 1,
 129: 0,
 48: 2,
 118: 0,
 47: 0,
 70: 0,
 77: 0,
 141: 0,
 81: 0,
 102: 0,
 125: 0,
 85: 0,
 25: 1,
 43: 1,
 101: 0,
 49: 1,
 5: 0,
 126: 1,
 93: 0,
 87: 0,
 91: 0,
 79: 0,
 26: 1,
 9: 1,
 28: 1,
 27: 1,
 84: 0,
 32: 1,
 137: 1,
 30: 1,
 94: 0,
 97: 0,
 104: 0,
 98: 0,
 41: 1,
 33: 1,
 103: 0,
 95: 0,
 96: 0,
 120: 0,
 44: 1,
 135: 0,
 14: 2,
 99: 0,
 35: 1,
 36: 1,
 38: 1,
 37: 0,
 1

## Hyperopt

In [28]:
from ax.service.managed_loop import optimize

In [29]:
from torch_geometric.nn import (
    HeteroConv,
    global_mean_pool,
    GATv2Conv
)
from torch.nn import (
    ModuleList,
    Module,
    Linear
  )
from typing_extensions import Self

In [30]:
class HGNN(Module):

    def __init__(self, output_cat, output_real,nodes_relations, relations_with_features, parameters) -> Self:  # type: ignore
        super().__init__()

        # List of convolutional layers
        
        hid = parameters["hid"]
        layers = parameters["layers"]
        aggregation = parameters["aggregation"]
        n_heads = parameters["heads"]
        
        self.output_cat = output_cat
        self.output_real = output_real
        
        self.convs = ModuleList()
        for _ in range(layers):
            conv = HeteroConv(
                {
                    relation: (
                        GATv2Conv((-1,-1), add_self_loops=False, out_channels=hid, heads=n_heads, concat=False)
                        if relation not in relations_with_features
                        else GATv2Conv((-1,-1), add_self_loops=False, out_channels=hid,heads=n_heads, edge_dim=relations_with_features[relation], concat=False)
                    )
                    for relation in nodes_relations
                },
                aggr=aggregation,
            )

            self.convs.append(conv)

        self.FC = {}
        
        for k in output_cat:
            self.FC[k] = Linear(hid, output_cat[k], device=device)
        for k in output_real:
            self.FC[k] = Linear(hid, 1, device=device)
        
    
        
        

    def forward(self, batch):

        for i in range(len(self.convs)):
            batch.x_dict = self.convs[i]( 
                batch.x_dict, batch.edge_index_dict, batch.edge_attr_dict
            )

            batch.x_dict = {key: x.relu() for key, x in batch.x_dict.items()}


        output = {}
        
        for k in self.output_cat:
            output[k] = global_mean_pool(batch.x_dict[k], batch[k].batch)
            output[k] = self.FC[k](output[k])
        for k in self.output_real:
            output[k] = global_mean_pool(batch.x_dict[k], batch[k].batch)
            output[k] = self.FC[k](output[k]).reshape(1,-1)[0]
            

        return output

In [31]:
from torcheval.metrics.functional import multiclass_accuracy, multiclass_f1_score

In [32]:
import torch.nn as nn

In [33]:
import time

In [34]:
from copy import deepcopy


def train_hgnn(config, output_cat, output_real, epochs=15):
    print(config)

    net = HGNN(
        parameters=config,
        output_cat=output_cat,
        output_real=output_real,
        nodes_relations=edge_types,
        relations_with_features=edge_features_dims,
    )
    net = net.to(device)

    losses = {}

    for k in output_cat:
        losses[k] = (
            nn.CrossEntropyLoss()
            if k != "Activity"
            else nn.CrossEntropyLoss(act_weights)
        )
    for k in output_real:
        losses[k] = nn.L1Loss()

    train_loader = DataLoader(X_train, batch_size=config["batch_size"], shuffle=True)
    valid_loader = DataLoader(X_valid, batch_size=config["batch_size"], shuffle=True)

    optimizer = torch.optim.Adam(net.parameters(), lr=config["lr"])

    best_model = None
    best_loss = 0
    patience = 4
    pat_count = 0

    torch.cuda.empty_cache()

    for epoch in range(0, epochs):
        start_time = time.time()

        print(f"Epoch: {epoch}\n")

        net.train()
        for _, x in enumerate(train_loader):
            x = x.to(device)

            labels = x.y

            optimizer.zero_grad()
            outputs = net(x)

            losses_step = {k: losses[k](outputs[k], labels[k]) for k in losses}

            total_loss = 0
            for k in losses_step:
                total_loss += losses_step[k]

            total_loss.backward()
            optimizer.step()

        predictions_categorical = {k: [] for k in output_cat}
        target_categorical = {k: [] for k in output_cat}

        avg_MAE = {k: [] for k in output_real}

        running_total_loss = []

        net.eval()
        with torch.no_grad():
            for i, x in enumerate(valid_loader):
                x = x.to(device)

                labels = x.y

                outputs = net(x)

                losses_step = {k: losses[k](outputs[k], labels[k]) for k in losses}

                running_total_loss.append(sum(list(losses_step.values())))

                for k in output_cat:
                    predictions_categorical[k].append(
                        torch.argmax(torch.softmax(outputs[k], dim=1), 1)
                    )
                    target_categorical[k].append(labels[k])

                for k in output_real:
                    avg_MAE[k].append(losses_step[k])

        for k in predictions_categorical:
            predictions_categorical[k] = torch.cat(predictions_categorical[k])
            target_categorical[k] = torch.cat(target_categorical[k])

        macro_f1_activity = multiclass_f1_score(
            predictions_categorical["Activity"],
            target_categorical["Activity"],
            num_classes=output_cat["Activity"],
            average="macro",
        )

        accuracy = {
            k: multiclass_accuracy(
                predictions_categorical[k],
                target_categorical[k],
                num_classes=output_cat[k],
            )
            for k in output_cat
        }

        avg_MAE = {k: sum(avg_MAE[k]) / len(avg_MAE[k]) for k in avg_MAE}

        val_loss = sum(running_total_loss) / len(running_total_loss)

        print(f"\nVALIDATION")
        for k in accuracy:
            (
                print("{}: acc {:.4f}".format(k, accuracy[k]))
                if k != "Activity"
                else print(
                    "{}: acc {:.4f} macroF1 {:.4f}".format(
                        k, accuracy[k], macro_f1_activity.item()
                    )
                )
            )
        for k in avg_MAE:
            print("{}: MAE {:.4f}".format(k, avg_MAE[k]))
        print("TOTAL_LOSS: {:.4f}".format(val_loss))
        print("epoch time {}s\n".format(time.time() - start_time))

        if epoch == 0:
            best_model = deepcopy(net)
            best_loss = val_loss
        else:
            if val_loss < best_loss:
                best_loss = val_loss
                best_model = deepcopy(net)
                pat_count = 0
                print("new best model found")
            if pat_count == patience:
                print(
                    "Validation performance didn't improve for {} epochs. Training stops.".format(
                        pat_count
                    )
                )
                return best_model
        pat_count += 1

    return best_model

In [40]:
def test_hgnn(net, output_cat, output_real):
    test_loader = DataLoader(X_train, batch_size=128, shuffle=False)
    
    losses = {}
    
    for k in output_cat:
        losses[k] = (
            nn.CrossEntropyLoss()
            if k != "Activity"
            else nn.CrossEntropyLoss(act_weights)
        )
    for k in output_real:
        losses[k] = nn.L1Loss()
    
    
    
    
    
    predictions_categorical = {k: [] for k in output_cat}
    target_categorical = {k: [] for k in output_cat}

    avg_MAE = {k : [] for k in output_real}
    
    total_loss = []
        
    net.eval()
    with torch.no_grad():
        for i, x in enumerate(test_loader):
            x = x.to(device)
            
            labels = x.y
            
            outputs = net(x)
            
     
            losses_step = {k: losses[k](outputs[k], labels[k]).item() for k in losses}
            total_loss.append(sum(list(losses_step.values())))
            
            for k in output_cat:
                    predictions_categorical[k].append(
                        torch.argmax(torch.softmax(outputs[k], dim=1), 1)
                    )
                    target_categorical[k].append(labels[k])
            
            
            for k in output_real:
                    avg_MAE[k].append(losses_step[k])
                    
    for k in predictions_categorical:
            predictions_categorical[k] = torch.cat(predictions_categorical[k])
            target_categorical[k] = torch.cat(target_categorical[k])
               
            
    macro_f1_activity = multiclass_f1_score(
            predictions_categorical["Activity"],
            target_categorical["Activity"],
            num_classes=output_cat["Activity"],
            average="macro",
        )
            
    accuracy = {
            k: multiclass_accuracy(
                predictions_categorical[k],
                target_categorical[k],
                num_classes=output_cat[k],
            )
            for k in output_cat
        }
    
    resource_to_role_acc = multiclass_accuracy(
        torch.tensor([map_resource_to_role[x.item()] for x in predictions_categorical["org:resource"]], device=device),
        target_categorical["org:resource:role"],
        num_classes=output_cat["org:resource:role"]
    )
    
    avg_MAE = {k : sum(avg_MAE[k]) / len(avg_MAE[k]) for k in avg_MAE}
    
    
    Average_total_loss = sum(total_loss) / len(total_loss)
    
    res = {f"{k}_acc" : accuracy[k].item() for k in accuracy} | {"Resource_to_role_acc" : resource_to_role_acc.item()} | {"MacroF1Act" : macro_f1_activity.item()} | {f"{k}_mae" : avg_MAE[k] for k in avg_MAE} | {"AVG_total_loss" : Average_total_loss} 
    
    print(res)
    
    return res

In [20]:
list_unique = {k : list(tab_all[k].unique()) for k in categorical_columns}
list_unique

{'Action': ['Created', 'statechange'],
 'org:resource': ['User_17',
  'User_1',
  'User_116',
  'User_95',
  'User_29',
  'User_119',
  'User_38',
  'User_68',
  'User_19',
  'User_90',
  'User_52',
  'User_117',
  'User_115',
  'User_3',
  'User_112',
  'User_87',
  'User_113',
  'User_27',
  'User_99',
  'User_118',
  'User_8',
  'User_75',
  'User_120',
  'User_16',
  'User_43',
  'User_89',
  'User_15',
  'User_28',
  'User_20',
  'User_7',
  'User_2',
  'User_30',
  'User_114',
  'User_14',
  'User_109',
  'User_9',
  'User_13',
  'User_24',
  'User_31',
  'User_100',
  'User_127',
  'User_5',
  'User_10',
  'User_70',
  'User_22',
  'User_11',
  'User_49',
  'User_92',
  'User_85',
  'User_4',
  'User_41',
  'User_71',
  'User_21',
  'User_37',
  'User_110',
  'User_44',
  'User_34',
  'User_74',
  'User_36',
  'User_18',
  'User_12',
  'User_108',
  'User_101',
  'User_6',
  'User_47',
  'User_35',
  'User_50',
  'User_25',
  'User_102',
  'User_33',
  'User_144',
  'User_26',
 

In [21]:
outputcat = {k : len(list_unique[k]) if len(list_unique[k]) > 1 else None for k in list_unique}
outputcat = {k : v for k,v in outputcat.items() if v != None}
outputreal = real_value_columns
print(outputcat)
print(outputreal)

{'Action': 2, 'org:resource': 144, 'Activity': 8, 'case:Selected': 2, 'case:Accepted': 2, 'org:resource:role': 3}
['time:timestamp', 'case:MonthlyCost', 'case:FirstWithdrawalAmount', 'case:CreditScore', 'case:OfferedAmount', 'case:NumberOfTerms']


In [41]:
def train_evaluate(config):
    trained_net = train_hgnn(config, output_cat=outputcat, output_real=outputreal, epochs = 15)
    return test_hgnn(trained_net, output_cat=outputcat, output_real=outputreal)

In [36]:
config = {'lr': 0.005059480954002739, 'batch_size': 256, 'heads': 1, 'hid': 128, 'layers': 2, 'aggregation': 'max'}

In [42]:
train_evaluate(config)

{'lr': 0.005059480954002739, 'batch_size': 256, 'heads': 1, 'hid': 128, 'layers': 2, 'aggregation': 'max'}
Epoch: 0




VALIDATION
Action: acc 1.0000
org:resource: acc 0.3293
Activity: acc 0.9123 macroF1 0.7375
case:Selected: acc 1.0000
case:Accepted: acc 1.0000
org:resource:role: acc 0.6431
time:timestamp: MAE 0.1655
case:MonthlyCost: MAE 0.1444
case:FirstWithdrawalAmount: MAE 0.1175
case:CreditScore: MAE 0.0802
case:OfferedAmount: MAE 0.1726
case:NumberOfTerms: MAE 0.1245
TOTAL_LOSS: 5.7062
epoch time 114.68652367591858s

Epoch: 1




VALIDATION
Action: acc 1.0000
org:resource: acc 0.3828
Activity: acc 0.6222 macroF1 0.6267
case:Selected: acc 1.0000
case:Accepted: acc 1.0000
org:resource:role: acc 0.9155
time:timestamp: MAE 0.0095
case:MonthlyCost: MAE 0.0517
case:FirstWithdrawalAmount: MAE 0.0626
case:CreditScore: MAE 0.0817
case:OfferedAmount: MAE 0.2322
case:NumberOfTerms: MAE 0.0959
TOTAL_LOSS: 4.8498
epoch time 107.0270619392395s

new best model found
Epoch: 2




VALIDATION
Action: acc 1.0000
org:resource: acc 0.4184
Activity: acc 0.7742 macroF1 0.7093
case:Selected: acc 1.0000
case:Accepted: acc 1.0000
org:resource:role: acc 0.9259
time:timestamp: MAE 0.0200
case:MonthlyCost: MAE 0.2030
case:FirstWithdrawalAmount: MAE 0.0519
case:CreditScore: MAE 0.0558
case:OfferedAmount: MAE 0.0333
case:NumberOfTerms: MAE 0.0971
TOTAL_LOSS: 4.4810
epoch time 111.10540103912354s

new best model found
Epoch: 3




VALIDATION
Action: acc 1.0000
org:resource: acc 0.4239
Activity: acc 0.8765 macroF1 0.7291
case:Selected: acc 1.0000
case:Accepted: acc 1.0000
org:resource:role: acc 0.9203
time:timestamp: MAE 0.0290
case:MonthlyCost: MAE 0.0323
case:FirstWithdrawalAmount: MAE 0.0362
case:CreditScore: MAE 0.0859
case:OfferedAmount: MAE 0.0398
case:NumberOfTerms: MAE 0.0713
TOTAL_LOSS: 4.2387
epoch time 107.36133766174316s

new best model found
Epoch: 4



...


VALIDATION
Action: acc 1.0000
org:resource: acc 0.4369
Activity: acc 0.8560 macroF1 0.7481
case:Selected: acc 1.0000
case:Accepted: acc 1.0000
org:resource:role: acc 0.9335
time:timestamp: MAE 0.0080
case:MonthlyCost: MAE 0.0691
case:FirstWithdrawalAmount: MAE 0.0823
case:CreditScore: MAE 0.0361
case:OfferedAmount: MAE 0.0269
case:NumberOfTerms: MAE 0.1390
TOTAL_LOSS: 4.1851
epoch time 108.4265308380127s

Epoch: 9




VALIDATION
Action: acc 1.0000
org:resource: acc 0.4049
Activity: acc 0.8824 macroF1 0.7522
case:Selected: acc 1.0000
case:Accepted: acc 1.0000
org:resource:role: acc 0.9311
time:timestamp: MAE 0.0194
case:MonthlyCost: MAE 0.0275
case:FirstWithdrawalAmount: MAE 0.0820
case:CreditScore: MAE 0.0312
case:OfferedAmount: MAE 0.0277
case:NumberOfTerms: MAE 0.0845
TOTAL_LOSS: 4.2344
epoch time 112.06352305412292s

Epoch: 10




VALIDATION
Action: acc 1.0000
org:resource: acc 0.4444
Activity: acc 0.7727 macroF1 0.7108
case:Selected: acc 1.0000
case:Accepted: acc 1.0000
org:resource:role: acc 0.9208
time:timestamp: MAE 0.0191
case:MonthlyCost: MAE 0.0598
case:FirstWithdrawalAmount: MAE 0.0809
case:CreditScore: MAE 0.1101
case:OfferedAmount: MAE 0.0894
case:NumberOfTerms: MAE 0.0880
TOTAL_LOSS: 4.2525
epoch time 107.41154384613037s

Validation performance didn't improve for 4 epochs. Training stops.


{'Action_acc': 0.9999999403953552, 'org:resource_acc': 0.48116424679756165, 'Activity_acc': 0.7834238409996033, 'case:Selected_acc': 0.9999999403953552, 'case:Accepted_acc': 0.9999999403953552, 'org:resource:role_acc': 0.9262759685516357, 'Resource_to_role_acc': 0.6959299445152283, 'MacroF1Act': 0.7208932638168335, 'time:timestamp_mae': 0.015734590044481585, 'case:MonthlyCost_mae': 0.020813669142352713, 'case:FirstWithdrawalAmount_mae': 0.05942680108421099, 'case:CreditScore_mae': 0.048451327766501295, 'case:OfferedAmount_mae': 0.07399701006826685, 'case:NumberOfTerms_mae': 0.046360369655562936, 'AVG_total_loss': 3.3774326820210274}


{'Action_acc': 0.9999999403953552,
 'org:resource_acc': 0.48116424679756165,
 'Activity_acc': 0.7834238409996033,
 'case:Selected_acc': 0.9999999403953552,
 'case:Accepted_acc': 0.9999999403953552,
 'org:resource:role_acc': 0.9262759685516357,
 'Resource_to_role_acc': 0.6959299445152283,
 'MacroF1Act': 0.7208932638168335,
 'time:timestamp_mae': 0.015734590044481585,
 'case:MonthlyCost_mae': 0.020813669142352713,
 'case:FirstWithdrawalAmount_mae': 0.05942680108421099,
 'case:CreditScore_mae': 0.048451327766501295,
 'case:OfferedAmount_mae': 0.07399701006826685,
 'case:NumberOfTerms_mae': 0.046360369655562936,
 'AVG_total_loss': 3.3774326820210274}

In [30]:
best_parameters, values, experiment, model = optimize(
    parameters=[
        {"name": "hid", "type": "choice", "values": [128], "value_type": "int", "is_ordered" : True,"sort_values":False},
        #{"name": "layers", "type": "choice", "values": [2, 3, 4, 5], "value_type": "int", "is_ordered" : True, "sort_values":False},
        {"name": "layers", "type": "choice", "values": [2], "value_type": "int", "is_ordered" : True, "sort_values":False},
        {"name": "lr", "type": "range", "bounds": [1e-4, 1e-1], "value_type": "float", "log_scale": True},
        {"name": "batch_size", "type": "choice", "values": [128, 256, 512], "value_type": "int", "is_ordered" : True,"sort_values":False}, 
        {"name": "heads", "type": "choice", "values": [1,2], "value_type": "int", "is_ordered" : True,"sort_values":False},
        #{"name": "heads", "type": "choice", "values": [1], "value_type": "int", "is_ordered" : True,"sort_values":False},
        
        #{"name": "aggregation", "type" : "choice", "values" :["sum", "mean", "max"], "value_type" : "str"}
        {"name": "aggregation", "type" : "choice", "values" :["max"], "value_type" : "str"},
     
    ],
  
    evaluation_function=train_evaluate,
    objective_name='AVG_total_loss',
    arms_per_trial=1,
    minimize = True,
    random_seed = 123,
    total_trials = 20
)

print(best_parameters)
means, covariances = values
print(means)
print(experiment)

[INFO 01-10 11:15:17] ax.service.utils.instantiation: Choice parameter hid contains only one value, converting to a fixed parameter instead.
[INFO 01-10 11:15:17] ax.service.utils.instantiation: Choice parameter layers contains only one value, converting to a fixed parameter instead.
[INFO 01-10 11:15:17] ax.service.utils.instantiation: Choice parameter aggregation contains only one value, converting to a fixed parameter instead.
[INFO 01-10 11:15:17] ax.service.utils.instantiation: Created search space: SearchSpace(parameters=[FixedParameter(name='hid', parameter_type=INT, value=128), FixedParameter(name='layers', parameter_type=INT, value=2), RangeParameter(name='lr', parameter_type=FLOAT, range=[0.0001, 0.1], log_scale=True), ChoiceParameter(name='batch_size', parameter_type=INT, values=[128, 256, 512], is_ordered=True, sort_values=False), ChoiceParameter(name='heads', parameter_type=INT, values=[1, 2], is_ordered=True, sort_values=False), FixedParameter(name='aggregation', paramete

{'lr': 0.0001865322718407359, 'batch_size': 256, 'heads': 1, 'hid': 128, 'layers': 2, 'aggregation': 'max'}
Epoch: 0




VALIDATION
Action: acc 1.0000
org:resource: acc 0.0941
Activity: acc 0.5414 macroF1 0.5078
case:Selected: acc 0.8374
case:Accepted: acc 0.7096
org:resource:role: acc 0.6342
time:timestamp: MAE 0.0802
case:MonthlyCost: MAE 0.3764
case:FirstWithdrawalAmount: MAE 2.1119
case:CreditScore: MAE 2.1796
case:OfferedAmount: MAE 0.5509
case:NumberOfTerms: MAE 0.3719
TOTAL_LOSS: 13.9506
epoch time 94.19226050376892s

Epoch: 1




VALIDATION
Action: acc 1.0000
org:resource: acc 0.0942
Activity: acc 0.8191 macroF1 0.6643
case:Selected: acc 0.8198
case:Accepted: acc 0.7096
org:resource:role: acc 0.6371
time:timestamp: MAE 0.0518
case:MonthlyCost: MAE 0.2949
case:FirstWithdrawalAmount: MAE 0.6630
case:CreditScore: MAE 0.2239
case:OfferedAmount: MAE 0.2336
case:NumberOfTerms: MAE 0.3408
TOTAL_LOSS: 9.3545
epoch time 96.94986701011658s

new best model found
Epoch: 2




VALIDATION
Action: acc 1.0000
org:resource: acc 0.0964
Activity: acc 0.7821 macroF1 0.6766
case:Selected: acc 0.8198
case:Accepted: acc 0.7096
org:resource:role: acc 0.7513
time:timestamp: MAE 0.0476
case:MonthlyCost: MAE 0.2245
case:FirstWithdrawalAmount: MAE 0.0722
case:CreditScore: MAE 0.0172
case:OfferedAmount: MAE 0.1644
case:NumberOfTerms: MAE 0.3024
TOTAL_LOSS: 7.8754
epoch time 96.42041063308716s

new best model found
Epoch: 3



...


VALIDATION
Action: acc 1.0000
org:resource: acc 0.4536
Activity: acc 0.7024 macroF1 0.6783
case:Selected: acc 1.0000
case:Accepted: acc 1.0000
org:resource:role: acc 0.9189
time:timestamp: MAE 0.0351
case:MonthlyCost: MAE 0.0600
case:FirstWithdrawalAmount: MAE 0.1256
case:CreditScore: MAE 0.0406
case:OfferedAmount: MAE 0.0207
case:NumberOfTerms: MAE 0.0195
TOTAL_LOSS: 3.9209
epoch time 93.70681548118591s

Epoch: 13




VALIDATION
Action: acc 1.0000
org:resource: acc 0.4486
Activity: acc 0.8174 macroF1 0.7335
case:Selected: acc 1.0000
case:Accepted: acc 1.0000
org:resource:role: acc 0.9354
time:timestamp: MAE 0.0102
case:MonthlyCost: MAE 0.0164
case:FirstWithdrawalAmount: MAE 0.1106
case:CreditScore: MAE 0.0695
case:OfferedAmount: MAE 0.0334
case:NumberOfTerms: MAE 0.1206
TOTAL_LOSS: 3.9865
epoch time 94.78157210350037s

Epoch: 14




VALIDATION
Action: acc 1.0000
org:resource: acc 0.4456
Activity: acc 0.6997 macroF1 0.6818
case:Selected: acc 1.0000
case:Accepted: acc 1.0000
org:resource:role: acc 0.9348
time:timestamp: MAE 0.0215
case:MonthlyCost: MAE 0.0463
case:FirstWithdrawalAmount: MAE 0.0463
case:CreditScore: MAE 0.0413
case:OfferedAmount: MAE 0.0280
case:NumberOfTerms: MAE 0.0155
TOTAL_LOSS: 3.8333
epoch time 96.09457302093506s

Validation performance didn't improve for 4 epochs. Training stops.


[INFO 01-10 20:35:55] ax.core.experiment: Attached data has some metrics ({'case:CreditScore_mae', 'case:Accepted_acc', 'case:OfferedAmount_mae', 'time:timestamp_mae', 'case:MonthlyCost_mae', 'Action_acc', 'org:resource_acc', 'case:NumberOfTerms_mae', 'case:Selected_acc', 'case:FirstWithdrawalAmount_mae', 'org:resource:role_acc', 'MacroF1Act', 'Activity_acc'}) that are not among the metrics on this experiment. Note that attaching data will not automatically add those metrics to the experiment. For these metrics to be automatically fetched by `experiment.fetch_data`, add them via `experiment.add_tracking_metric` or update the experiment's optimization config.
[ERROR 01-10 20:35:55] ax.core.observation: Data contains metric Action_acc that has not been added to the experiment. You can either update the `optimization_config` or attach it as a tracking metric using `Experiment.add_tracking_metrics` or `AxClient.add_tracking_metrics`. Ignoring all data for metric Action_acc.
NoneType: None


{'lr': 0.004552086573166931, 'batch_size': 256, 'heads': 1, 'hid': 128, 'layers': 2, 'aggregation': 'max'}
{'Action_acc': 0.9999999403953552, 'org:resource_acc': 0.5017408132553101, 'Activity_acc': 0.6825777888298035, 'case:Selected_acc': 0.9999999403953552, 'case:Accepted_acc': 0.9999999403953552, 'org:resource:role_acc': 0.9231773018836975, 'MacroF1Act': 0.6557358503341675, 'time:timestamp_mae': 0.02948719680823701, 'case:MonthlyCost_mae': 0.01615671095394743, 'case:FirstWithdrawalAmount_mae': 0.03462876550405222, 'case:CreditScore_mae': 0.021030696332952493, 'case:OfferedAmount_mae': 0.020618635273324887, 'case:NumberOfTerms_mae': 0.023892845829869964, 'AVG_total_loss': 3.0018922934759193}
Experiment(None)


In [31]:
from ax.service.utils.report_utils import exp_to_df

results = exp_to_df(experiment)

[WARNING 01-10 20:39:44] ax.service.utils.report_utils: Column reason missing for all trials. Not appending column.


In [36]:
results.sort_values(by="AVG_total_loss")

,trial_index,arm_name,trial_status,generation_method,AVG_total_loss,Action_acc,Activity_acc,MacroF1Act,case:Accepted_acc,case:CreditScore_mae,case:FirstWithdrawalAmount_mae,case:MonthlyCost_mae,case:NumberOfTerms_mae,case:OfferedAmount_mae,case:Selected_acc,org:resource:role_acc,org:resource_acc,time:timestamp_mae,lr,batch_size,heads,hid,layers,aggregation
17,17,17_0,COMPLETED,BoTorch,3.001892,1.0,0.682578,0.655736,1.000000,0.021031,0.034629,0.016157,0.023893,0.020619,1.000000,0.923177,0.501741,0.029487,0.004552,256,1,128,2,max
18,18,18_0,COMPLETED,BoTorch,3.021418,1.0,0.872345,0.753679,1.000000,0.035895,0.040981,0.035529,0.013212,0.028054,1.000000,0.925910,0.500644,0.018920,0.005059,256,1,128,2,max
3,3,3_0,COMPLETED,Sobol,3.097343,1.0,0.827519,0.739030,1.000000,0.013569,0.031308,0.033009,0.020665,0.023483,1.000000,0.926119,0.498625,0.029681,0.002055,256,2,128,2,max
9,9,9_0,COMPLETED,BoTorch,3.108473,1.0,0.916284,0.771517,1.000000,0.036862,0.020158,0.073669,0.025462,0.018249,1.000000,0.925963,0.505240,0.017166,0.002888,256,1,128,2,max
6,6,6_0,COMPLETED,BoTorch,3.111150,1.0,0.805933,0.729211,1.000000,0.018861,0.020971,0.013464,0.016587,0.040519,1.000000,0.920253,0.496379,0.019450,0.004245,256,2,128,2,max
19,19,19_0,COMPLETED,BoTorch,3.120008,1.0,0.900912,0.760799,1.000000,0.031455,0.013704,0.013744,0.035916,0.034613,1.000000,0.925284,0.496849,0.016809,0.002324,256,1,128,2,max
14,14,14_0,COMPLETED,BoTorch,3.134812,1.0,0.867854,0.753487,1.000000,0.032583,0.031117,0.034165,0.009901,0.026596,1.000000,0.927599,0.502524,0.017660,0.007438,256,1,128,2,max
1,1,1_0,COMPLETED,Sobol,3.243917,1.0,0.650059,0.651953,1.000000,0.016081,0.031671,0.031044,0.017071,0.030560,1.000000,0.925858,0.487762,0.040250,0.020423,128,2,128,2,max
5,5,5_0,COMPLETED,Sobol,3.303718,1.0,0.837181,0.739332,1.000000,0.011582,0.014421,0.015416,0.018535,0.043731,1.000000,0.924779,0.481895,0.017765,0.003913,128,1,128,2,max
16,16,16_0,COMPLETED,BoTorch,3.349779,1.0,0.830670,0.735465,1.000000,0.022075,0.099470,0.010681,0.061138,0.019711,1.000000,0.928173,0.479702,0.014093,0.008139,128,1,128,2,max


In [32]:
results = results.sort_values(by="AVG_total_loss")

In [33]:
results.to_csv("results/BPI17O.csv", sep=",")